In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [9]:
df_xgbm = pd.read_pickle(f'{folder_path}/oof_stacking_xgbm_tss5', compression='gzip').rename(columns={'isFraud':'xgbm_isFraud'})

In [3]:
df_oof_train = pd.DataFrame()
df_oof_test = pd.DataFrame()

df_oof_vae_train = pd.read_pickle(f'{folder_path}/oof_vae_train', compression='gzip')
df_oof_vae_test = pd.read_pickle(f'{folder_path}/oof_vae_test', compression='gzip')
df_oof_train = df_oof_vae_train
df_oof_test = df_oof_vae_test

df_oof_lgbm_train = pd.read_pickle(f'{folder_path}/oof_lgbm_train', compression='gzip')
df_oof_lgbm_test = pd.read_pickle(f'{folder_path}/oof_lgbm_test', compression='gzip')
df_oof_lgbm_test['predict'] = df_oof_lgbm_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_lgbm_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_lgbm'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_lgbm_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_lgbm'}), how='left', on='TransactionID')

df_oof_xgbm_train = pd.read_pickle(f'{folder_path}/oof_xgbm_train', compression='gzip')
df_oof_xgbm_test = pd.read_pickle(f'{folder_path}/oof_xgbm_test', compression='gzip')
df_oof_xgbm_test['predict'] = df_oof_xgbm_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_xgbm_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_xgbm'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_xgbm_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_xgbm'}), how='left', on='TransactionID')

df_oof_mlp_train = pd.read_pickle(f'{folder_path}/oof_mlp_train', compression='gzip')
df_oof_mlp_test = pd.read_pickle(f'{folder_path}/oof_mlp_test', compression='gzip')
df_oof_mlp_test['predict'] = df_oof_mlp_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_mlp_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_mlp'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_mlp_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_mlp'}), how='left', on='TransactionID')

df_oof_1dcnn_train = pd.read_pickle(f'{folder_path}/oof_1dcnn_train', compression='gzip')
df_oof_1dcnn_test = pd.read_pickle(f'{folder_path}/oof_1dcnn_test', compression='gzip')
# df_oof_1dcnn_test['predict'] = df_oof_1dcnn_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_1dcnn_train[['TransactionID', 'isFraud']].rename(columns={'isFraud':'oof_1dcnn'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_1dcnn_test[['TransactionID', 'isFraud']].rename(columns={'isFraud':'oof_1dcnn'}), how='left', on='TransactionID')

In [4]:
df_oof_test.head()

,TransactionID,loss,kle,reconstructione,mse,oof_lgbm,oof_xgbm,oof_mlp,oof_1dcnn
0,3663549,76.213234,0.131185,76.082048,0.170198,0.000186,0.000334,0.000497,0.000324
1,3663550,83.605424,0.125992,83.479432,0.199555,0.001328,0.002672,0.000377,0.002861
2,3663551,80.684792,0.125003,80.559789,0.155993,0.001123,0.000482,0.001226,0.002663
3,3663552,80.136013,0.127257,80.008757,0.156172,0.000416,0.001683,0.004226,0.001190
4,3663553,78.773938,0.126601,78.647337,0.149714,0.000441,0.000414,0.011174,0.003880


In [5]:
df_oof_test['isFraud'] = df_oof_test[['oof_lgbm','oof_xgbm','oof_mlp','oof_1dcnn']].mean(axis=1)

In [6]:
df_oof_test.head()

,TransactionID,loss,kle,reconstructione,mse,oof_lgbm,oof_xgbm,oof_mlp,oof_1dcnn,isFraud
0,3663549,76.213234,0.131185,76.082048,0.170198,0.000186,0.000334,0.000497,0.000324,0.000335
1,3663550,83.605424,0.125992,83.479432,0.199555,0.001328,0.002672,0.000377,0.002861,0.001810
2,3663551,80.684792,0.125003,80.559789,0.155993,0.001123,0.000482,0.001226,0.002663,0.001373
3,3663552,80.136013,0.127257,80.008757,0.156172,0.000416,0.001683,0.004226,0.001190,0.001879
4,3663553,78.773938,0.126601,78.647337,0.149714,0.000441,0.000414,0.011174,0.003880,0.003977


In [7]:
df_oof_test[['TransactionID', 'isFraud']].to_csv(f'{folder_path}/blending_mean.csv',index=False)